In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from google.colab import drive
import cv2
import matplotlib.pyplot as plt
import os.path as osp 
import pickle
import numpy as np

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition/test_image/'
img2 = cv2.imread("tree.jpg", cv2.IMREAD_COLOR)
plt.imshow(img2[:,:,::-1])

In [ ]:
%%time
%cd '/content/'
!cp '/content/drive/My Drive/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition/data/ntu/zipped/transformed_data.tgz' 'transformed_data.tgz'
%cd '/content/'
!tar -xf 'transformed_data.tgz' -C './'

In [ ]:
%%time
save_path = osp.join('/content/ntu/', 'transformed_data')
evaluation = 'C-Subject'
x_pkl = osp.join(save_path, '%s_x.pkl' % (evaluation))
y_pkl = osp.join(save_path, '%s_y.pkl' % (evaluation))
with open(x_pkl, 'rb') as f:
    train_X = pickle.load(f)
with open(y_pkl, 'rb') as f:
    train_Y = pickle.load(f)

CPU times: user 747 ms, sys: 2.76 s, total: 3.51 s
Wall time: 24.6 s


In [ ]:
print('train_X.shape', train_X.shape, 'train_X[0].shape = ',  train_X[0].shape)
print('train_Y.shape =', train_Y.shape)
#train_Y includes 59874 samples and each sample is categorized into one of 120 action classes. 
print('max(train_Y) = ', max(train_Y), 'min(train_Y) =', min(train_Y))
print('np.sum(train_Y==5) = ', np.sum(train_Y==5))
print('np.sum(train_Y==119) = ', np.sum(train_Y==119))
print('train_Y[:20] = ', train_Y[:20])
#train_X includes (59874 samples, 57 frames, 150 2 people joint information)
#150 joints(3D locations of 25 major body joints=75 * 2 people) == 150
print("train_X[0][0][:20] = ", train_X[0][0][:20])

detail description of the file

For each frame of a skeleton sequence, an actor's 3D positions of 25 joints represented by an 2D array (shape: 25 x 3) is reshaped into a 75-dim vector by concatenating each 3-dim (x, y, z) coordinates along the row dimension in joint order. Each frame contains two actor's joints positions constituting a 150-dim vector. If there is only one actor, then the last 75 values are filled with zeros. Otherwise, select the main actor and the second actor based on the motion amount. Each 150-dim vector as a row vector is put into a 2D numpy array where the number of rows equals the number of valid frames. All such 2D arrays are put into a list and finally the list is serialized into a cPickle file. For the skeleton sequence which contains two or more actors (mostly corresponds to the last 11 classes), the filename and actors' information are recorded into log files. For better understanding, also generate RGB+skeleton videos for visualization.

use .pkl file, 0 means setting  is CV, 1 means the other setting because in each batch, different samples have different sequence_length. padded using 0 to make every sample has max_sequence_length. Then, you need len.pkl to know the originial length of the sequence for each sample 

att is attention = (batch_size , sequence_length , 150)
len is length of each sample in batch = (batch_size, )
x is inputs = (batch_size , sequence_length , 150)
y is labels = (batch_size, )

In [ ]:
%%time
%cd '/content/'
!cp '/content/drive/My Drive/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition/data/ntu/zipped/results_debug1.tgz' 'results_debug1.tgz'
%cd '/content/'
!tar -xf 'results_debug1.tgz' -C './'

/content
/content
CPU times: user 36.4 ms, sys: 15.4 ms, total: 51.9 ms
Wall time: 12.6 s


In [ ]:
save_path = osp.join('/content/ntu/', 'results_debug1')

In [ ]:
ls results_debug1

0_att.pkl    0_len.pkl   0_x.pkl    1_best.pth   1_log.csv   1_y.pkl
0_best.pth   0_log.csv   0_y.pkl    1_label.txt  1_pred.txt  result.txt
0_label.txt  0_pred.txt  1_att.pkl  1_len.pkl    1_x.pkl


In [ ]:
with open('results_debug1/0_x.pkl', 'rb') as f:
  skeleton_cv_crd = pickle.load(f)
with open('results_debug1/0_y.pkl', 'rb') as f:
  skeleton_cv_label = pickle.load(f)
with open('results_debug1/0_att.pkl', 'rb') as f:
  skeleton_cv_att = pickle.load(f)
with open('results_debug1/1_x.pkl', 'rb') as f:
  skeleton_cs_crd = pickle.load(f)
with open('results_debug1/1_y.pkl', 'rb') as f:
  skeleton_cs_label = pickle.load(f)
with open('results_debug1/1_att.pkl', 'rb') as f:
  skeleton_cs_att = pickle.load(f)

A23. hand waving.
A34. rub two hands together.
A36. shake head.
A44. touch head (headache).


A52. pushing other person.
A56. giving something to other person
A58. handshaking.

In [ ]:
# 1 person action
# A23==>22. hand waving. (good)
# A34==>33. rub two hands together.
# A35==>34. nod head/bow. (good)
# A36==>35. shake head. (not good)
# A44==>43. touch head (good)
# 2 people action
# A52==>51. pushing other person.
# A56==>55. giving something to other person
# A58==>57. handshaking.

candidates = [22, 33, 34, 35, 43, 51, 55, 57]
print('cv case')
for idx, value in enumerate(skeleton_cv_label):
  if value in candidates:
    print("idx = {}, value ={}".format(idx, value))

print('cs case')
for idx, value in enumerate(skeleton_cs_label):
  if value in candidates:
    print("idx = {}, value ={}".format(idx, value))

In [ ]:
with open('results_debug1/0_label.txt', 'rb') as f:
  for i in range(100):
    print(f.readline())

In [ ]:
print('skeleton_cv_crd.shape ={}, skeleton_cv_label.shape={}, skeleton_cv_att.shape={}'.format(skeleton_cv_crd.shape, skeleton_cv_label.shape, skeleton_cv_att.shape))

skeleton_cv_crd.shape =(256, 233, 150), skeleton_cv_label.shape=(256,), skeleton_cv_att.shape=(256, 233, 150)


In [ ]:
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math
from IPython import display
from time import sleep

%cd '/content/drive/My Drive/Computer_Vision_CSE527/project/jongwoo_rnn_vis/'
img1 = cv2.imread('fruits.jpg', cv2.IMREAD_COLOR)
plt.imshow(img1[:,:,::-1])

SsssCcccPpppRrrrAaaa (e.g., S001C002P003R002A013), in which sss is the setup number, ccc is the camera ID, ppp is the performer (subject) ID, rrr is the replication number (1 or 2), and aaa is the action class label.

In [ ]:
trunk_joints = [0, 1, 20, 2, 3]
arm_joints = [23, 24, 11, 10, 9, 8, 20, 4, 5, 6, 7, 22, 21]
leg_joints = [19, 18, 17, 16, 0, 12, 13, 14, 15]
skeletons_in_body = [trunk_joints, arm_joints, leg_joints]

# skeleton animation for NTU-3D
class Draw3DSkeleton(object):    
    def __init__(self, file, save_path=None, init_horizon=-45,
                 init_vertical=20, x_rotation=None,
                 y_rotation=None, pause_step=0.2, is_file_txt=True):

        self.save_path = save_path
        self.arranged_xyz = None
        self.xyz = None
        self.file = None

        #file is inserted
        if is_file_txt:
          self.file = file
          if not os.path.exists(self.save_path):
              os.mkdir(self.save_path)
          #  self.xyz[#coordinate, #frames, #joints, #people]
          self.xyz = self.read_xyz(self.file)
          self.num_coordinate = self.xyz.shape[0]
          self.num_frame = self.xyz.shape[1]
          self.num_joints = self.xyz.shape[2]
          self.num_people = self.xyz.shape[3]

          self.init_horizon = init_horizon
          self.init_vertical = init_vertical

          self.x_rotation = x_rotation
          self.y_rotation = y_rotation

          self._pause_step = pause_step

          self.relative_response = None
          self.scaled_relative_response = None
        #array is inserted
        else:
          #self.arranged_xyz should have [body, #frames, #joints, #coordinate]
          #file has [seq, 150 coordinates]
          body1 = data_from_net[:,:75]
          body2 = data_from_net[:,75:]
          body1 = body1.reshape(-1, 25, 3)
          body2 = body2.reshape(-1, 25, 3)
          empty_body = np.zeros(body2.shape)
          if np.allclose(body2, empty_body): 
            print('body2 is empty')
            self.arranged_xyz = body1[np.newaxis, ...]
          else:           
            print('body2 exists')
            self.arranged_xyz = np.concatenate((body1, body2), axis=0).reshape(2, -1, 25, 3)
          print('arranged_xyz.shape = ', self.arranged_xyz.shape)
          self.num_coordinate = self.arranged_xyz.shape[3]
          self.num_frame = self.arranged_xyz.shape[1]
          self.num_joints = self.arranged_xyz.shape[2]
          self.num_people = self.arranged_xyz.shape[0]

          self.init_horizon = init_horizon
          self.init_vertical = init_vertical

          self.x_rotation = x_rotation
          self.y_rotation = y_rotation

          self._pause_step = pause_step

          self.relative_response = None
          self.scaled_relative_response = None


    def _read_skeleton(self, file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            #81 frames (it looks correct)
            # print('skeleton_sequence[numFrame] =' , skeleton_sequence['numFrame'])
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []
                for m in range(frame_info['numBody']):
                    body_info = {}
                    #numbers on the first line. 72057594037940044 0 1 1 1 1 0 -0.07373063 -0.1171718 2
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)
        # print("skeleton_sequence['frameInfo'][30]", skeleton_sequence['frameInfo'][30])        
        # print("skeleton_sequence['frameInfo'][29]['bodyInfo'][0]['jointInfo'][11]", skeleton_sequence['frameInfo'][29]['bodyInfo'][0]['jointInfo'][11])
        # print("skeleton_sequence['frameInfo'][30]['bodyInfo'][0]['jointInfo'][11]", skeleton_sequence['frameInfo'][30]['bodyInfo'][0]['jointInfo'][11])        
        return skeleton_sequence

    #simple_data has a format of [sequence_length , 150]
    def read_xyz(self, file, max_body=2, num_joint=25):
        seq_info = self._read_skeleton(file)
        # print('numBody = ', seq_info['frameInfo'][0]['numBody'])
        data = np.zeros((3, seq_info['numFrame'], num_joint, seq_info['frameInfo'][0]['numBody']))  # (3(#xyz), frame_nums, 25(#joints), 2(#people))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[:, n, j, m] = [v['x'], v['y'], v['z']]
                    else:
                        pass
        # check data exists in 31st frame, 12th joint, 1st body.
        # data[#coordinate, #frames, #joints, #body]
        # print('data[:,30, 17, 0]', data[:,30, 17, 0])
        return data    

    def _normal_skeleton(self, data):
        #data is in the form of data[#body, #frames, #joints, #coordinate]
        #  use as center joint
        # center_joint is an array of #body, #frame, #coordinate for 0th joint
        center_joint = data[:, :, 0, :]

        center_jointx = np.mean(center_joint[:,:, 0] )
        center_jointy = np.mean(center_joint[:,:, 1])
        center_jointz = np.mean(center_joint[:,:, 2])

        center = np.array([center_jointx, center_jointy, center_jointz])
        data = data - center

        return data

    def _rotation(self, data, alpha=0, beta=0):
        # rotate the skeleton around x-y axis
        r_alpha = alpha * np.pi / 180
        r_beta = beta * np.pi / 180

        rx = np.array([[1, 0, 0],
                       [0, np.cos(r_alpha), -1 * np.sin(r_alpha)],
                       [0, np.sin(r_alpha), np.cos(r_alpha)]]
                      )

        ry = np.array([
            [np.cos(r_beta), 0, np.sin(r_beta)],
            [0, 1, 0],
            [-1 * np.sin(r_beta), 0, np.cos(r_beta)],
        ])

        r = ry.dot(rx)
        data = data.dot(r)

        return data

    def set_relative_response(self, attention_response=None, arrange_required=False):
      #data_xyz is in the shape of [#body, #frames, #coordinate, #joints]
      if self.arranged_xyz is not None:
        data_xyz = np.transpose(self.arranged_xyz, (0,1,3,2))  #arranged_xyz is [#body, #frames, #joints, #coordinate]
      if self.arranged_xyz is None:        
        data_xyz = np.transpose(self.xyz, (3, 1, 0, 2))
      if attention_response is None:
        attention_response = np.ones([self.num_people, self.num_frame, self.num_joints, self.num_coordinate])
      #set attention_response in the form of [#body, #frames, #joints, #coordinate] from [#frame, 150 coordinates]
      if arrange_required == True:
        body1 = attention_response[:,:75]
        body2 = attention_response[:,75:]
        body1 = body1.reshape(-1, 25, 3)
        body2 = body2.reshape(-1, 25, 3)
        empty_body = np.zeros(body2.shape)
        if np.allclose(body2, empty_body): 
          print('body2 is empty in attention_response')
          attention_response = body1[np.newaxis, ...]
        else:           
          print('body2 exists in attention_response')
          attention_response = np.concatenate((body1, body2), axis=0).reshape(2, -1, 25, 3)
      # print('data_xyz[#body, #frames, #coordinate, #joints] = \n', data_xyz, '\n')
      data_xyz_center = data_xyz[...,0]
      # print('data_xyz_center = \n', data_xyz_center, '\n')
      data_xyz_center_diff = data_xyz - data_xyz_center[..., np.newaxis]
      # print('data_xyz_center_diff = \n', data_xyz_center_diff, '\n')
      energy = np.power(data_xyz_center_diff, 2)
      # print('energy [#body, #frames, #coordinate, #joints]= \n', energy, '\n')
      energy_diff_abs = np.abs(np.diff(energy, axis=1))
      # print('energy_diff_abs = \n', energy_diff_abs, '\n')
      energy_diff_abs_mean = np.mean(energy_diff_abs, axis=3)
      # print('energy_diff_abs_mean = \n', energy_diff_abs_mean, '\n')
      energy_diff_abs_mean_shape = energy_diff_abs_mean.shape
      energy_ones = np.ones((energy_diff_abs_mean_shape[0], 1, energy_diff_abs_mean_shape[2]))
      #weight_response is in the shape of[#body, #frames, #coordinate]
      weight_response = np.append(energy_ones, energy_diff_abs_mean, axis=1)
      # print("weight_response =\n", weight_response, '\n')
      #attention_response is in the form of [#body, #frames, #joints, #coordinate]
      #relative_response_xyz is in the form of[#body, #frames, #joints, #coordinate]
      relative_response_xyz = attention_response / weight_response[..., np.newaxis, :]
      # print('relative_response_xyz = \n', relative_response_xyz, '\n')
      #self.relative_response is in a shape of [#person, #frames, #joint]
      self.relative_response = np.sum(relative_response_xyz, axis=3)
      # print('self.relative_response = \n', self.relative_response, '\n')

      # # this is the case where 0-1 scale on each person. relative_response_max is in a shape of [#person, #frames]
      # relative_response_max = np.amax(self.relative_response, axis=2)

      # this is the case where 0-1 scale on both people. relative_response_max is in a shape of [#frames]
      relative_response_max = np.amax(np.amax(self.relative_response, axis=2), axis=0)

      #self.scaled_relative_response is in a shape of [#person, #frames, #joint].It provides 0-1 scaled relative response
      self.scaled_relative_response = self.relative_response / relative_response_max[np.newaxis, : , np.newaxis]

    def visual_skeleton_animate(self, use_relative_response=False, scattersize_max=10, frame_idx_total=None, sleep_time=0.5, figsize=(10,10), \
                                is_image_save=True, skeleton_color='r', joint_color='yellow'):
        #data becomes data[#body, #frames, #joints, #coordinate] from [#coordinate, #frames, #joints, #body]
        if self.arranged_xyz is not None:
          data = self.arranged_xyz
        if self.arranged_xyz is None:        
          data = np.transpose(self.xyz, (3, 1, 2, 0))
        
        num_people = data.shape[0]
        # print('num_people =', num_people)
        fig = plt.figure(figsize=figsize)
        axs = fig.add_subplot(projection='3d')

        # data rotation
        if (self.x_rotation is not None) or (self.y_rotation is not None):

            if self.x_rotation > 180 or self.y_rotation > 180:
                raise Exception("rotation angle should be less than 180.")

            else:
                data = self._rotation(data, self.x_rotation, self.y_rotation)

        # data normalization
        data = self._normal_skeleton(data)

        data_flatten_xyz = data.reshape((-1,3))
        xyz_max = np.amax(data_flatten_xyz, axis=0)
        xyz_min = np.amin(data_flatten_xyz, axis=0)
        xz_range = [np.amin(xyz_min[[0,2]], axis=None), np.amax(xyz_max[[0,2]], axis=None)]
        y_range =  [xyz_min[1], xyz_max[1]]
        xyz_range = [np.amin(xyz_min, axis=None), np.amax(xyz_max, axis=None)]

        # show every frame 3d skeleton
        # realsimulation
        if frame_idx_total is None:
          frame_idx_total = range(data.shape[1])
        else:
          frame_idx_total = frame_idx_total
        
        idx_axs = 1
        test_z_min = 999
        test_x_plot = None
        test_y_plot = None
        test_z_plot = None
        for frame_idx in frame_idx_total:
            axs.set_facecolor("none")
            #range should be adjusted
            #the screen length of x,y,z are same regardless of your xlim, ylim, zlim. Therefore, it is important to put the smiliar values in xlim, ylim, zlim to keep the ratio 1:1:1.
            axs.set_xlim3d(xyz_range)
            # here, ylim3d corresponds to z (depth) in the data coordinate (the graph label it as Z)
            axs.set_ylim3d(xyz_range)
            # here, zlim3d corresponds to y (height) in the data coordinate (the graph label it as Y)
            # you replace the min of zlim to actual y_range[0] to keep the feet of people on the floor
            axs.set_zlim3d([y_range[0], xyz_range[1]])

            #define ticks              
            tick_perturb = 0.01
            # axs.set_xticks(np.arange(-1 - tick_perturb, 0.5 + tick_perturb, step=0.5))
            # axs.set_yticks(np.arange(-0.5 - tick_perturb, 1 + tick_perturb, step=0.5))
            # axs.set_zticks(np.arange(-0.8 - tick_perturb, 0.8 + tick_perturb, step=0.4))

            # for each_person in range(1):
            for each_person in range(num_people):
              #data is in shape of [#of people, #frames, #joints, coordinte]
              x = data[each_person, frame_idx, :, 0]
              y = data[each_person, frame_idx, :, 1]
              z = data[each_person, frame_idx, :, 2]
              # print("frame_idx={}, x={},x.size={}, \n y={}, y.size={}, \n z={},z.size={}".format(frame_idx, x, x.size, y, y.size, z, z.size))

              #drawing skeletons_in_body in one axs
              for idx_part, part in enumerate(skeletons_in_body):
                  #each part is trunk, arm, leg
                  #x_plot is x values in trunk or arm or leg. array of numbers 
                  x_plot = x[part]
                  y_plot = y[part]
                  z_plot = z[part]
                  if test_z_min > np.amin(z_plot):
                    test_z_min = np.amin(z_plot)
                  if np.amin(z_plot) < -0.336:
                    test_x_plot = x_plot
                    test_y_plot = y_plot
                    test_z_plot = z_plot
                  if use_relative_response == False or self.relative_response is None:
                    #trunk case. increase the circle size
                    if idx_part ==0:
                      scattersize_joint = np.ones(x_plot.size) * 10 * scattersize_max
                    else:
                      scattersize_joint = np.ones(x_plot.size) * scattersize_max
                  #formula for relative_response to scattersize_joint
                  else:
                    #self.scaled_relative_response provides 0-1 scaled relative response
                    #scattersize_joint is in shape of [# of joints]
                    scattersize_joint = self.scaled_relative_response[each_person, frame_idx, part] * scattersize_max
                  # the ratio of elements in scattersize_joint is the ratio of relative response
                  # print('scattersize_joint = ', scattersize_joint)
                  axs.plot(x_plot, z_plot, y_plot, color=skeleton_color, marker='o', markersize=0.1, markerfacecolor=joint_color)
                  axs.scatter(x_plot, z_plot, y_plot, s=scattersize_joint, c=joint_color)

            #decorate the axs
            axs.set_xlabel('X')
            axs.set_ylabel('Z')
            axs.set_zlabel('Y')
            axs.set_title('{} frame'.format(frame_idx))
            if (self.save_path is not None) and (is_image_save == True):
                # print statement interrupt the figure. So no use if you want to animate the figure 
                # print("save_path frame_idx = ", frame_idx)
                save_pth = os.path.join(self.save_path, '{}.png'.format(frame_idx))
                plt.savefig(save_pth)
                save_check_period = 0.01
                while (not(os.path.exists(save_pth))):
                  sleep(save_check_period)
                with open(save_pth, 'r') as skeleton_file:
                  #  print('passed both os.path.exists and open(file)')
                  pass
                # print('save_pth = {}, png saved? = {}'.format(save_pth, os.path.exists(save_pth)))
            #animation part
            display.display(plt.gcf())            
            sleep(sleep_time)
            #prepare the next plot
            #remove the current axs for the next plot
            if (frame_idx != frame_idx_total[-1]):
              display.clear_output(wait=True)
              plt.cla()
            idx_axs += 1
        
        #after showing plot of each frame in animation, you only leave the last frame image. So plt.clf() will remove the regualr axs.plot() image coming at the last
        plt.clf()
        # print('test_z_min = ', test_z_min)
        # print('test_x_plot = ', test_x_plot)
        # print('test_y_plot = ', test_y_plot)
        # print('test_z_plot = ', test_z_plot)
        # print('xyz_max = ', xyz_max)
        # print('xyz_min = ', xyz_min)
        # print('xz_range =', xz_range)
        # print('y_range =', y_range)

#??? I am not sure the scattersize becomes the ratio of area or volume
#??? maybe draw skeleton in 2D and show the relative_response


# New Section

In [ ]:
#################################### test test #################################################
#################################### test test #################################################
#################################### test test #################################################
#################################### test test #################################################
class test_visual():
  def __init__(self, xyz):
    #self.xyz is in the shape of [#coordinate, #frames, #joints, #body]
    self.xyz = xyz
    self.num_coordinate = self.xyz.shape[0]
    self.num_frame = self.xyz.shape[1]
    self.num_joints = self.xyz.shape[2]
    self.num_people = self.xyz.shape[3]
    self.relative_response =None
    self.scaled_relative_response = None

  def set_relative_response(self, attention_response=None):
    #data_xyz is in the shape of [#body, #frames, #coordinate, #joints]
    data_xyz = np.transpose(self.xyz, (3, 1, 0, 2))
    if attention_response is None:
      attention_response = np.ones([self.num_people, self.num_frame, self.num_joints, self.num_coordinate])
    print('data_xyz[#body, #frames, #coordinate, #joints] = \n', data_xyz, '\n')
    data_xyz_center = data_xyz[...,0]
    print('data_xyz_center = \n', data_xyz_center, '\n')
    data_xyz_center_diff = data_xyz - data_xyz_center[..., np.newaxis]
    print('data_xyz_center_diff = \n', data_xyz_center_diff, '\n')
    energy = np.power(data_xyz_center_diff, 2)
    print('energy [#body, #frames, #coordinate, #joints]= \n', energy, '\n')
    energy_diff_abs = np.abs(np.diff(energy, axis=1))
    print('energy_diff_abs = \n', energy_diff_abs, '\n')
    energy_diff_abs_mean = np.mean(energy_diff_abs, axis=3)
    print('energy_diff_abs_mean = \n', energy_diff_abs_mean, '\n')
    energy_diff_abs_mean_shape = energy_diff_abs_mean.shape
    energy_ones = np.ones((energy_diff_abs_mean_shape[0], 1, energy_diff_abs_mean_shape[2]))
    #weight_response is in the shape of[#body, #frames, #coordinate]
    weight_response = np.append(energy_ones, energy_diff_abs_mean, axis=1)
    print("weight_response =\n", weight_response, '\n')
    #attention_response is in the form of [#body, #frames, #joints, #coordinate]
    #relative_response_xyz is in the form of[#body, #frames, #joints, #coordinate]
    relative_response_xyz = attention_response / weight_response[..., np.newaxis, :]
    print('relative_response_xyz = \n', relative_response_xyz, '\n')
    #self.relative_response is in a shape of [#person, #frames, #joint]
    self.relative_response = np.sum(relative_response_xyz, axis=3)
    print('self.relative_response = \n', self.relative_response, '\n')

    # # this is the case where 0-1 scale on each person. relative_response_max is in a shape of [#person, #frames]
    # relative_response_max = np.amax(self.relative_response, axis=2)

    # this is the case where 0-1 scale on both people. relative_response_max is in a shape of [#frames]
    relative_response_max = np.amax(np.amax(self.relative_response, axis=2), axis=0)

    #self.scaled_relative_response is in a shape of [#person, #frames, #joint].It provides 0-1 scaled relative response
    self.scaled_relative_response = self.relative_response / relative_response_max[np.newaxis, : , np.newaxis]


In [ ]:
#self.xyz is in the shape of [#coordinate, #frames, #joints, #body]
# test_xyz = np.random.rand(3, 3, 3, 2)
#used to find out the weight
test_xyz = np.ones((3, 3, 3, 2))
test_xyz[:,1,:,:]=2
test_xyz[:,:,0,:]=3
print('test_xyz = \n', test_xyz)
#attention_response is in the form of [#body, #frames, #joints, #coordinate]
# attention_response = np.ones((2, 3, 3, 3))
# attention_response = [[[[1, 1, 1,],
#    [1, 1, 1,],
#    [1, 1, 1,]],

#   [[1, 1, 1,],
#    [1, 1, 1,],
#    [1, 1, 1,]],

#   [[1, 1, 1,],
#    [1, 1, 1,],
#    [1, 1, 1,]]],


#  [[[100, 10, 10,],
#    [10, 100, 10,],
#    [10, 10, 100,]],

#   [[100, 10, 10,],
#    [100, 10, 10,],
#    [100, 10, 10,]],

#   [[10, 10, 100,],
#    [10, 10, 100,],
#    [10, 10, 100,]]]]

attention_response = [[[[1, 1, 1,],
   [1, 1, 1,],
   [1, 1, 1,]],

  [[1, 1, 1,],
   [1, 1, 1,],
   [1, 1, 1,]],

  [[1, 1, 1,],
   [1, 1, 1,],
   [1, 1, 1,]]],


 [[[1, 1, 1,],
   [1, 1, 1,],
   [1, 1, 1,]],

  [[1, 1, 1,],
   [1, 1, 1,],
   [1, 1, 1,]],

  [[1, 1, 1,],
   [1, 1, 1,],
   [1, 1, 1,]]]]

print('attention_response = \n', attention_response, '\n')
test_a = test_visual(test_xyz)
test_a.set_relative_response(attention_response)
#self.relative_response is in a shape of [#person, #frames, #joint] from [#body, #frames, #joints, #coordinate]
#self.scaled_relative_response is in a shape of [#person, #frames, #joint]
print('test_a.scaled_relative_response = \n', test_a.scaled_relative_response)
#let's test more cases

In [ ]:
#self.xyz is in the shape of [#coordinate, #frames, #joints, #body]
# test_xyz = np.random.rand(3, 3, 3, 2)
#used to find out the weight
test_xyz = np.ones((3, 3, 3, 2))
test_xyz[:,1,:,:]=2
test_xyz[:,:,0,:]=3
print('test_xyz = \n', test_xyz)
#attention_response is in the form of [#body, #frames, #joints, #coordinate]
# attention_response = np.ones((2, 3, 3, 3))
test_a = test_visual(test_xyz)
test_a.set_relative_response()
#self.relative_response is in a shape of [#person, #frames, #joint] from [#body, #frames, #joints, #coordinate]
#self.scaled_relative_response is in a shape of [#person, #frames, #joint]
print('test_a.scaled_relative_response = \n', test_a.scaled_relative_response)
#let's test more cases

In [ ]:
test_a = np.array([[[1,2,3],
                    [4,5,6]],
                   [[7,8,9],
                    [10,11,12]]])
test_b = test_a.reshape((-1, 3))
test_c = np.ones((2,1,3))
print('test_a = \n,', test_a, '\n')
print('test_a.shape = ', test_a.shape)
print('test_b = \n', test_b, '\n')
print('test_c = \n', test_c, '\n')
test_d = np.append(test_c, test_a, axis=1)
print('test_d (test_a + test_c) = \n', test_d, '\n')
print('test_d.shape = \n', test_d.shape)
print('amax =', np.amax(np.amax(test_a, axis=0), axis=0))
print('amin =', np.amin(np.amin(test_a, axis=0), axis=0))
print('amax =', np.amax(test_b, axis=None))

In [ ]:
test_a = np.array([[[1,2,3],
                    [4,5,6]],
                   [[7,8,9],
                    [1,2,3]]])
test_c = np.ones((2,1,3))
print('test_a = \n,', test_a, '\n')
print('test_a.shape = ', test_a.shape)
print('test_c = \n', test_c, '\n')
test_d = np.append(test_c, test_a, axis=1)
print('test_d (test_a + test_c) = \n', test_d, '\n')
print('test_d.shape = \n', test_d.shape)

test_energy = np.diff(test_a, axis=1)
print('test_energy = \n', test_energy, '\n test_energy.shape= ', test_energy.shape)
test_energy_appended = np.append(test_c, test_energy, axis=1)
print('test_energy_appended = \n', test_energy_appended)
test_energy_mean = np.mean(test_energy_appended, axis=2)
print('test_energy_mean \n', test_energy_mean)

print('np.power(test_a, 2) = \n', np.power(test_a, 2))
test_e = test_a[..., 0]
print('test_e = \n', test_e, 'test_e.shape = ', test_e.shape)

test_f = test_a - test_e[..., np.newaxis]
print('test_f = \n', test_f, 'test_f.shape = ', test_f.shape)
print('test_a = \n,', test_a, '\n')



In [ ]:
#2 people data
sk = Draw3DSkeleton("S001C001P001R001A060.skeleton", './test_result')

relative_response = np.ones(25)
trunk_joints = np.array([0, 1, 20, 2, 3])
relative_response[trunk_joints] = 10

# frame_idx_total = np.array([4, 50, 70])
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, frame_idx_total=frame_idx_total, sleep_time=0.2)

# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5)

sk.visual_skeleton_animate(use_relative_response=False, scattersize_max=10, sleep_time=0.5, is_image_save = False )

In [ ]:
#2 people data
sk = Draw3DSkeleton("S001C001P001R001A060.skeleton", './test_result')
#attention_response is in the form of [#body, #frames, #joints, #coordinate]
attention_response = sk.set_relative_response(attention_response=None)
sk.visual_skeleton_animate(use_relative_response=True, scattersize_max=50, sleep_time=0.5, is_image_save = True )

In [ ]:
sk = Draw3DSkeleton("S001C001P001R002A035.skeleton", './test_result')
# sk.visual_skeleton(relative_response=None, scattersize_max=10)
relative_response = np.ones(25)
trunk_joints = np.array([0, 1, 20, 2, 3])
relative_response[trunk_joints] = 10

#method1
# frame_idx_total = np.array([4, 50, 70])
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, frame_idx_total=frame_idx_total, sleep_time=0.2)
#method2
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5)
#method3
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5, is_image_save = False )
#method4 using attention response
attention_response = sk.set_relative_response(attention_response=None)
sk.visual_skeleton_animate(use_relative_response=True, scattersize_max=50, sleep_time=0.5, is_image_save=True)

In [ ]:
data_from_net = skeleton_cv_crd[34]
body1 = data_from_net[:,:75]
body2 = data_from_net[:,75:]
body1 = body1.reshape(-1, 25, 3)
print(body1.shape)
body2 = body2.reshape(-1, 25, 3)
print(body2.shape)
empty_body = np.zeros(body2.shape)
print(empty_body.shape)
if np.allclose(body2, empty_body): 
  print('body2 is empty')
  arranged_xyz = body1[np.newaxis, ...]
else:           
  print('body2 exists')
  arranged_xyz = np.concatenate((body1, body2), axis=0).reshape(2, -1, 25, 3)
print('arranged_xyz.shape = ', arranged_xyz.shape)

(233, 25, 3)
(233, 25, 3)
(233, 25, 3)
body2 is empty
arranged_xyz.shape =  (1, 233, 25, 3)


In [ ]:
################### use the data from AttG-GRU ######################
################### use the data from AttG-GRU ######################
################### use the data from AttG-GRU ######################
cut_frame_idx = 200
# sample_idx = 214  #bow
sample_idx = 56
path_test_result = './test_result'
path_rest_result_plain = './test_result_plain'
#cv case
data_from_net = skeleton_cv_crd[sample_idx][:cut_frame_idx] #no more frame after 60.
#cs case
# data_from_net = skeleton_cs_crd[sample_idx][:cut_frame_idx] #no more frame after 60.
sk = Draw3DSkeleton(file=data_from_net, save_path=path_test_result, is_file_txt=False)

# use the aritifical relative response
# relative_response = np.ones(25)
# trunk_joints = np.array([0, 1, 20, 2, 3])
# relative_response[trunk_joints] = 10

#use generated relative response
attention_response = skeleton_cv_att[sample_idx][:cut_frame_idx]

sk_plain = Draw3DSkeleton(file=attention_response, save_path=path_rest_result_plain, is_file_txt=False)

#method1
# frame_idx_total = np.array([4, 50, 70])
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, frame_idx_total=frame_idx_total, sleep_time=0.2)
#method2
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5)
#method3: using artificial relative response (make head big or something like that)
# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5, is_image_save = False )
#method4: using generated attention response
# sk.set_relative_response(attention_response=attention_response, arrange_required=True)
# sk.visual_skeleton_animate(use_relative_response=True, scattersize_max=200, sleep_time=0.5, is_image_save=True)
#method5: using generated attention response
sk.set_relative_response(attention_response=attention_response, arrange_required=True)
sk.visual_skeleton_animate(use_relative_response=True, scattersize_max=300, sleep_time=0.1, is_image_save=True, skeleton_color='r', joint_color='blue')
#method6: using aritificial all one response 
# sk_plain.set_relative_response(attention_response=None, arrange_required=False)
# sk_plain.visual_skeleton_animate(use_relative_response=True, scattersize_max=50, sleep_time=0.5, is_image_save=True, skeleton_color='r', joint_color='blue')

In [ ]:
sk = Draw3DSkeleton("S001C001P004R002A008.skeleton", './test_result')
# sk.visual_skeleton(relative_response=None, scattersize_max=10)
relative_response = np.ones(25)
trunk_joints = np.array([0, 1, 20, 2, 3])
relative_response[trunk_joints] = 10

frame_idx_total = np.array([4,14,34])
sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, frame_idx_total=frame_idx_total, sleep_time=0.2)

# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5)

# sk.visual_skeleton_animate(relative_response=relative_response, scattersize_max=100, sleep_time=0.5, is_image_save = False )

In [ ]:
# sk = Draw3DSkeleton("S001C001P004R002A008.skeleton", './test_result')
# # sk.visual_skeleton(relative_response=None, scattersize_max=10)
# relative_response = np.ones(25)
# trunk_joints = np.array([0, 1, 20, 2, 3])
# relative_response[trunk_joints] = 10
# frame_idx_total = np.array([1, 10, 20, 30, 40, 50])
# sk.visual_skeleton(relative_response=relative_response, scattersize_max=100, frame_idx_total=frame_idx_total)

In [ ]:

    # default horizon, vertical test 
    # sk2 = Draw3DSkeleton("S001C001P004R002A008.skeleton", './test_result', init_horizon=0, init_vertical=0)
    # sk2.visual_skeleton()
    # sk3 = Draw3DSkeleton("S001C001P004R002A008.skeleton", './test_result', init_horizon=-45, init_vertical=0)
    # sk3.visual_skeleton()
    # sk4 = Draw3DSkeleton("S001C001P004R002A008.skeleton", './test_result', init_horizon=0, init_vertical=20)
    # sk4.visual_skeleton()

In [ ]:
x = np.array([1,2,3])
y = np.array([-1,0,1])
markersize_const = 8
markersize_array = np.array([5,10,20]) * markersize_const
print('markersize_array = ', markersize_array)
fig, ax = plt.subplots(1,1,figsize=(14,7))
ax.plot(x,y, marker='o', markersize=0.1)
# plt.xticks([1,2,3])
ax.set_xticks(np.arange(x[0], x[-1]+0.01, step=0.3))
ax.set_yticks(np.arange(y[0], y[-1]+0.01, step=0.3))
ax.scatter(x,y, s=markersize_array, c='yellow')
for i in range(50):
  ax.scatter(x[2],y[2], s=markersize_array[2], c='yellow')

In [ ]:
x_array = np.array(range(10))
y_array = np.array(range(10))*-1 +5
z_array = np.array(range(10))
plt.ion()
fig = plt.figure(figsize=(14,7))
axs = fig.add_subplot(2,2,1, projection='3d')
axs.plot(x_array, y_array, z_array)
axs = fig.add_subplot(2,2,2, projection='3d')
axs.plot(y_array, x_array, z_array)
axs = fig.add_subplot(2,2,3, projection='3d')
axs.plot(z_array, y_array, x_array)

In [ ]:
from IPython import display
from time import sleep
x_array = np.array(range(10))
y_array = np.array(range(10))*-1+5
z_array = np.array(range(10))*-2+5
fig = plt.figure(figsize=(14,7))
axs = fig.add_subplot(projection='3d')
axs.plot(x_array, y_array, z_array)
print('111')
display.display(plt.gcf())
print('aaa')
sleep(2)
plt.cla()
display.clear_output(wait=True)
axs.plot(y_array, x_array, z_array)
display.display(plt.gcf())
print('bbb')
sleep(2)
# plt.clf()
print('ccc')
#if therre is no plt.clf(), axs.plot will be shown at the end of the execution. Therefore, the plot will appear after print('ccc')


# display.clear_output(wait=True)
# sleep(2)
# display.clear_output()
# axs.plot(z_array, y_array, x_array)

# display.display(axs.plot(x_array, y_array, z_array))
# display.clear_output()
# sleep(1)
# display.display(axs.plot(y_array, x_array, z_array))
# display.clear_output()
# axs = fig.add_subplot(2,2,3, projection='3d')
# axs.plot(z_array, y_array, x_array)